# Data Anonymizer

This notebook anonymizes the log files. It processes grades to associated them to the log files.

### Imports

In [1]:
import os
import pandas as pd
import random

### File and Config Variables

We define all input and output filenames as well as other parameters.

In [2]:
# name of the module where the logfile originates from
#   RT - Reutlingen
#   WIB - Wirtschaftsinformatik Bachelor
#   Stat| Net | SE - Statistik | Computernetzwerke | Software Engineering
#   W22 | SS23 - Winter term, summer term and year
subject = 'RTWIBNet_W22'

# input filepaths
input_filepath = '/GradePredictionData/data'
output_filepath = '/GradePredictionData/data-registry'

# other Params
HSRT_IP_start = '134.103'

In [3]:
# input files
log_filepath = os.path.join(input_filepath, subject + '_log.csv')
grade_filepath = os.path.join(input_filepath, subject + '_grades.csv')

# lookup tables for names and ips; serve as input and get updated, if necessary
lookup_names_filepath = os.path.join(input_filepath, subject+'_names.csv')
lookup_ips_filepath = os.path.join(input_filepath, subject+'_ips.csv')

# anonymized files
logfile_output = os.path.join(output_filepath, subject, subject+'_log.csv')
grades_output = os.path.join(output_filepath, subject, subject+'_grades.csv')

### Load raw data

We load the raw data from `.csv` files into dataframes and assert required columns.

In [4]:
# load logfile, if missing, notebook will stop here
logfile = pd.read_csv(log_filepath, sep=',')
required_columns = ['Vollständiger Name', 'Betroffene/r Nutzer/in', 'IP-Adresse']
assert all([c for c in required_columns if c in list(logfile.columns)])
assert len(logfile) > 0

In [5]:
# load grades if exists, otherwise create empty dataframe
required_columns = ['nachname', 'vorname', 'bewertung']
try:
    grades = pd.read_csv(grade_filepath, sep=';')
except:
    grades = pd.DataFrame(data=[], columns=required_columns)
assert all([c for c in required_columns if c in list(grades.columns)])

In [6]:
# load lookup tables for names and ips, if they exist
## all_names
required_columns = ['Vollständiger Name', 'name_code']
try:
    all_names = pd.read_csv(lookup_names_filepath, sep=',')
except:
    all_names = pd.DataFrame(data=[], columns=required_columns)
assert all([c for c in required_columns if c in list(all_names.columns)])

## all_ips
required_columns = ['IP-Adresse', 'ip_code']
try:
    all_ips = pd.read_csv(lookup_ips_filepath, sep=',')
except:
    all_ips = pd.DataFrame(data=[], columns=required_columns)
assert all([c for c in required_columns if c in list(all_ips.columns)])

In [7]:
# for assertion after en-coding of names and IPs
input_all_names_length = len(all_names)
input_all_ips_length = len(all_ips)

### Some Filtering

In [8]:
# remove myself from logfile
logfile.drop(logfile[logfile['Vollständiger Name'] == 'Christian Decker'].index, inplace=True)
logfile.dropna(axis=0, how='any', inplace=True)

In [9]:
# Build complete name
grades['Vollständiger Name']= grades['vorname'] + ' ' + grades['nachname']

### Create Name Codes

Name code is a 4digit hex corresponding to the index in a dataframe of names. The names are shuffled beforehand to avoid information leak by sorted names. It creates an `all_names` dataframe with corresponding name / code entries, one per row. If `all_names` already exists, it will be updated by new names from the logfile or grade file.

In [10]:
# extract all names from logfile
logfile_names = set(list(logfile['Vollständiger Name']))
logfile_names = logfile_names.union(set(list(logfile['Betroffene/r Nutzer/in'])))
# extract all names from grades
grades_names = set(list(grades['Vollständiger Name']))
# combine both
name_list = list(logfile_names.union(grades_names))

In [11]:
# new_names_list contains new names, which are so far not part of names_list
# if name from names_list is not in all_names, it becomes part of new_names_list
new_names_list = [name for name in name_list if not (all_names['Vollständiger Name'].eq(name)).any()]

In [12]:
# shuffle to avoid lexigographic order
random.shuffle(new_names_list)
# create data frame with index continuing the all_names index (works with empty new_names_list, too)
start_idx = len(all_names)
end_idx = len(all_names) + len(new_names_list)
idx = list(range(start_idx, end_idx))
new_names = pd.DataFrame(new_names_list, index=idx, columns=['Vollständiger Name'])
# update all_names dataframe with new_names
all_names = pd.concat([all_names, new_names], axis='index')

In [13]:
# replace name by index in 4digit hex
all_names['name_code'] = all_names.apply(lambda row : f'{row.name:0>4X}' , axis=1)
# all_names must not decrease 
assert len(all_names) >= input_all_names_length

### Create IP Address Codes

Address code is a 4digit hex corresponding to the index in a dataframe of IP addresses, see `def ip_code(..)` for further details. The IPs are shuffled beforehand to avoid information leak by sorted IPs. It creates an `all_ips` dataframe with corresponding IP / code entries, one per row. If `all_ips` already exists, it will be updated by new IPs from the logfile.

In [14]:
# extract all IPs from logfile
ip_list = list(set(list(logfile['IP-Adresse'])))

In [15]:
# new_ip_list contains ips, which are not part existing all_ips
new_ip_list = [ip for ip in ip_list if not (all_ips['IP-Adresse'].eq(ip)).any()]

In [16]:
# shuffle and add to all_ips dataframe
random.shuffle(new_ip_list)
# create data frame with index continuing the all_ips index (works with empty new_ip_list, too)
start_idx = len(all_ips)
end_idx = len(all_ips) + len(new_ip_list)
idx = list(range(start_idx, end_idx))
new_ips = pd.DataFrame(new_ip_list, index=idx, columns=['IP-Adresse'])
# update all_names dataframe with new_names
all_ips = pd.concat([all_ips, new_ips], axis='index')

In [17]:
# returns 134.13.0.0 if IP originates from HSRT_IP_start
# or 4digit hex corresponding to a given index 
def ip_code(ip, idx):
    if ip.startswith(HSRT_IP_start):
        return HSRT_IP_start + '.0.0'
    else:
        return f'{idx:0>4X}'

In [18]:
# replace name by index in 4digit hex
all_ips['ip_code'] = all_ips.apply(lambda row : ip_code(str(row['IP-Adresse']), row.name) , axis=1)
# all_ips must not decrease
assert len(all_ips) >= input_all_ips_length

### Anonymize Names and IPs

In [19]:
# Lookup functions: provide name or IP and retrieve anonymized code
def anon_name(name):
    code = all_names[all_names['Vollständiger Name'] == name]['name_code']
    return code.iloc[0]

def anon_ip(ip):
    code = all_ips[all_ips['IP-Adresse'] == ip]['ip_code']
    return code.iloc[0]

In [20]:
# anonymize logfile
logfile['Vollständiger Name'] = logfile.apply(lambda row : anon_name(row['Vollständiger Name']) , axis = 1)
logfile['Betroffene/r Nutzer/in'] = logfile.apply(lambda row : anon_name(row['Betroffene/r Nutzer/in']) , axis = 1)
logfile['IP-Adresse'] = logfile.apply(lambda row : anon_ip(str(row['IP-Adresse'])) , axis = 1)

In [21]:
# anonymize grades
if len(grades) > 0: # we have grades
    grades['Vollständiger Name'] = grades.apply(lambda row : anon_name(row['Vollständiger Name']) , axis=1)
grades = grades[['Vollständiger Name', 'bewertung']]

### Store Anonymized Data as `.csv` File

In [22]:
# create directory, if not exists
data_reg_dir = os.path.dirname(logfile_output)
!mkdir -p {data_reg_dir}

In [23]:
logfile.to_csv(logfile_output, sep=',', index=False)
grades.to_csv(grades_output, sep=',', index=False)

### Store Lookup Tables for Names and IPs

This is useful for later use, when we want to keep the lookup tables while getting updated data.

In [24]:
all_names.to_csv(lookup_names_filepath, sep=',', index=False)
all_ips.to_csv(lookup_ips_filepath, sep=',', index=False)